# More on Piping, Intentions, and Column Expressions

In [3]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

In [5]:
artists = pd.read_csv("./data/Artists.csv")
artwork = pd.read_csv("./data/Artworks.csv")

In [7]:
# carried over from the last lecture
bad_lbls = (artists >> 
             filter_by(X.Nationality.str.lower().str.startswith('nation').astype('bool')) >>
             pull('Nationality')).unique()
recode_bad_lbls = {old_lbl:'Nationality unknown' for old_lbl in bad_lbls}
replace_zero = {0:np.NaN}

## Why we love piping? 

### Reason 1: Composition Baby!

It is very easy to put separate pipe together.

In [8]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID'))
artists_new = (artists >>
                mutate(Nationality = X.Nationality.replace(recode_bad_lbls)))
artists_new = (artists_renamed >>
                mutate(BeginDate = X.BeginDate.replace(replace_zero)))

## To compose separate pipes

1. Switch ending `)` to `>>`
2. Remove the next assignment
3. ??
4. Profit!

In [9]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') >> #)
#artists_new = (artists >>
                mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >> #)
#artists_new = (artists_renamed >>
                mutate(BeginDate = X.BeginDate.replace(replace_zero)))

## End product ... full process in a single pipe

In [10]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') >>
                    mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >>
                    mutate(BeginDate = X.BeginDate.replace(replace_zero)))

## Why we love piping? 

### Reason 2: Textual Gravity!

A pipe clearly expression the intention of our code by

1. Reading left-to-right and top-to-bottom
2. Putting the verbs up front

In [11]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') >>
                    mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >>
                    mutate(BeginDate = X.BeginDate.replace(replace_zero)))

## Why we love piping? 

### Reason 3: Easy debugging

Comments make it easy to debug a pipe.

## Debugging Step 1 - Start at the top

Use comments to remove all part of the chain

*Don't forget the ending `)`*

In [12]:
artists_renamed = (artists ) #>>
                    #rename(Wiki_QID = 'Wiki QID') >>
                    #mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >>
                    #mutate(BeginDate = X.BeginDate.replace(replace_zero)))

## Debugging Step 2 - Work your way down the pipe

Add in each part, one-at-a-time, checking the results

*Don't forget the ending `)`*

In [13]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') ) #>>
                    #mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >>
                    #mutate(BeginDate = X.BeginDate.replace(replace_zero)))

In [14]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') >>
                    mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) ) #>>
                    #mutate(BeginDate = X.BeginDate.replace(replace_zero)))

In [15]:
artists_renamed = (artists >>
                    rename(Wiki_QID = 'Wiki QID') >>
                    mutate(Nationality = X.Nationality.replace(recode_bad_lbls)) >>
                    mutate(BeginDate = X.BeginDate.replace(replace_zero)))

# More about Intentions 

## `X` is an `Intention`

<img src="img/dfply_X_intention_1.png" width = 400>

Think of it as recording an expression for later evaluation

In [16]:
expr = X.BeginDate.head()
expr

## Use `evaluate` to apply the expression

We can apply an expression *later* using `evaluate` with a dataframe.

In [17]:
expr.evaluate(artists)

0    1930
1    1936
2    1941
3    1946
4    1941
Name: BeginDate, dtype: int64

## Intention expressions are reusable!

In [18]:
# Reusable!
expr.evaluate(artwork)

0    (1841)
1    (1944)
2    (1876)
3    (1944)
4    (1876)
Name: BeginDate, dtype: object

## <font color="red"> Exercise 1 </font>
    
**Task:** Use `X` to create an expression that replaces spaces in column names with underscores.  Apply this expression to fresh instances of `artists` and `artwork`.

## Not just for data frames ... works for any* expression

In [19]:
double, line = 2*X, 3*X + 5

In [20]:
double.evaluate(3), line.evaluate(6)

(6, 23)

## We can make functions lazy too!

Decorate a function with `make_symbolic` to allow lazy evaluation of `Intention` objects

In [21]:
from math import log
log = make_symbolic(log)

In [22]:
log(8, 2) # Works as expected with numbers

3.0

## Passing in `X` now makes a `log` expression

In [23]:
expr = log(X, 2) # Passing in X makes it lazy/symbolic
expr

In [37]:
expr.evaluate(16) # Evaluate later

4.0

## `pyspark.sql` set up

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, column, mean

spark1 = SparkSession.builder.appName('Ops').getOrCreate()

## `pyspark.sql` columns also generate expression 

In [28]:
column('height')

Column<b'height'>

In [38]:
column('height') > 3

Column<b'(height > 3)'>

## `col == column`

In [41]:
5*col('height') + 2 # col is an alias for column

Column<b'((height * 5) + 2)'>

## Column expressions can combine columns

In [42]:
col('height') + col('weight')

Column<b'(height + weight)'>

## Columns work with other `pyspark.sql.functions`

In [43]:
mean(col('height'))

Column<b'avg(height)'>

## `sqlalchemy` columns generate expression too

In [66]:
from sqlalchemy import func as f
f.column('height')

<sqlalchemy.sql.functions.Function at 0x10f72abe0; column>

In [67]:
f.column('height') > 3

## `col == column`

In [68]:
5*f.col('height') + 2 # col is an alias for column

## Column expressions can combine columns

In [69]:
f.col('height') + f.col('weight')

## Columns work with other `pyspark.sql.functions`

In [71]:
f.avg(col('height'))

<sqlalchemy.sql.functions.Function at 0x10f758630; avg>

## Up Next

Now we will continue on to [Lecture 2.4](./2_4_pandas_dtypes.ipynb).
